In [4]:
#tupleX = tuple(x for x in tupleX if condition)

test_tuple = tuple(('a', 'b', 'c', 'd', 'e'))
ls = ['a', 'b']

new_tuple = tuple(x for x in test_tuple if x not in ls)
new_tuple

('c', 'd', 'e')

In [1]:
def random_combination_2(iterable, r, players): 
    #Random selection from combinations
    #Remove player combinations from player list
    iterable = [x for x in iterable if x not in players]
    
    iter_count = 0 
    combo_salary = 0
    salary_ls = []
    while combo_salary < 49000 or combo_salary > 50000:
        iter_count += 1
        if iter_count > 60000:
            if players[0] == 'placeholder':
                print('placeholder error')
                
            else:
                print('combination impossible')
                
            return salary_ls
        
        if r == 5:
            combo_salary = salary_dict[players[0]]
        elif r == 4:
            combo_salary = salary_dict[players[0]] + salary_dict[players[1]]
        else:
            combo_salary = 0
        
        pool = tuple(iterable)
        n = len(pool)
        indices = sorted(random.sample(range(n), r))
        team = tuple(pool[i] for i in indices)
        
        for t in team:
            combo_salary += salary_dict[t]
        
        salary_ls.append(tuple((team, combo_salary)))
        
    return tuple(pool[i] for i in indices)

def team_sim(team_ls):
    indices = np.arange(len(contest_array)).reshape(len(contest_array), 1)
    team_payout_ls = []
    
    team_ids_ls =[]
    for team in team_ls:
        
        team_ids = []

        for player in team:
            for key in pid_dict.keys():
                if pid_dict[key]['name'] == player:
                    team_ids.append(key)
        
        team_ids_ls.append(team_ids)
        
    lineup_payout_array = np.arange(len(team_ls)).reshape(-1,1)
    
    for i in tqdm(range(1,proj_array.shape[1])):

        #create dict of points 
        point_dict = dict(zip(proj_array[:,0], proj_array[:,i]))

        #select random int for replacement in contest array
        random_ls = []
        for team in team_ls:
            random_replacement = random.randint(0, len(contest_array) - 1)
            random_ls.append(random_replacement)

        #create_temp array for replacement
        temp_contest_array = contest_array
        
        for t, team_id in enumerate(team_ids_ls):
            random_num = random_ls[t]
            temp_contest_array[random_replacement] = team_id

        #create sim array of points from single simulation
        sim_array = np.vectorize(point_dict.__getitem__)(temp_contest_array)

        #sum up team points for simulation
        total_points = sim_array.sum(axis = 1).reshape((len(sim_array),1))

        #create add indicies to total points
        result_array = np.hstack((indices, total_points))

        #sort result array by points decending
        result_array = result_array[result_array[:, 1].argsort()[::-1]]

        #add payout array to results array
        result_array = np.hstack((result_array, payout_array))

        #revert result array back to being order by index
        result_array = result_array[result_array[:,0].argsort()]

        #append team result to team payout array
        iteration_payout_array = result_array[random_ls, -1].reshape(-1,1)
        lineup_payout_array = np.hstack((lineup_payout_array, iteration_payout_array))
        
           
    return lineup_payout_array

In [ ]:
num_lineups_per_player = 150
n = num_lineups_per_player
master_team_ls = []

def find_first_player(top_players, num_players_to_return):
    for p, player in enumerate(top_players):
        team_ls = []
        for i in range(num_lineups_per_player):
            team_ls.append(random_combination_2(players, 5, [player]) + tuple((player,)))

        master_team_ls.append(team_ls)

        result_array = team_sim(team_ls)
        result_array = np.hstack(((np.full((num_lineups_per_player, 1), p)), result_array))

        if p == 0:
            player_result_array = result_array
        else:
            player_result_array = np.vstack((player_result_array, result_array))

    result_dict = {}
    for p, player in enumerate(top_players):
        temp_array = player_result_array[p * n:p * n + n]
        result_dict[player] = {'total': np.sum(temp_array)}

    result_df = pd.DataFrame()
    result_df['players'] = result_dict.keys()
    result_df['return'] = result_df['players'].apply(lambda x: result_df[x]['total'])
    result_df.sort_values('return', ascending = False, inplace = True)
    result_df = results_df.head(num_players_to_return)
    
    return result_df['players'].values

def find_second_player(top_players, first_player, num_combos_to_return):
    for p, player in enumerate(top_players):
        team_ls = []
        for i in range(num_lineups_per_player):
            team_ls.append(random_combination_2(top_players, 4, [first_player, player]) + tuple((first_player, player)))
            
        master_team_ls.append(team_ls)
        
        result_array = team_sim(team_ls)
        result_array = np.hstack(((np.full((num_lineups_per_player, 1), p)), result_array))
        
        player_result_array = np.vstack((player_result_array, result_array))
        
    for p, player in enumerate(top_players):
        temp_array = player_result_array[p * n: p* n + n]
        combo_name = first_player + ', ' + player
        result_dict[combo_name] = {'total': np.sum(temp_array)}
        
    result_df = pd.DataFrame()
    result_df['combos'] = result_dict.keys()
    result_df['return'] = result_df['combos'].apply(lambda x: result_df[x]['total'])
    result_df.sort_values('return', ascending = False, inplace = True)
    result_df = results_df.head(num_players_to_return)
    
    return result_df['combos'].values
        
        

top_players = find_first_player(players[:10], 1)
top_2_man_combos = find_second_player(players[:30], top_players[0], 1)

    
